In [2]:
import numpy as np

In [ ]:
# def initialisation(X):
#     W = np.random.randn(X.shape[1], 1)
#     b = np.random.randn(1)
#     return (W, b)

![init](multi_layer_png/init.png "init")

In [5]:
def initialisation(n0, n1, n2):
    W1 = np.random.rand(n1, n0)
    b1 = np.random.randn(n1, 1)
    W2 = np.random.rand(n2, n1)
    b2 = np.random.randn(n2, 1)

    parametres = {
        'W1' : W1,
        'b1' : b1,
        'W2' : W2,
        'b2' : b2,
    }
    return parametres

In [ ]:
# def model(X, W, b):
#     Z = X.dot(W) + b
#     A = 1 / (1 + np.exp(-Z))
#     return A

In [ ]:
def forward_propagation(X, W, b):
    Z = X @ B + b
    A = 1 / (1 + np.exp(-Z))
    return A
# 3:08